In [ ]:
import os
import json
import numpy as np
import random

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


folders = ["Clickbait_Auto", "Clickbait_Direct", "NonClickbait_Auto"]
# folders = ["Clickbait_Direct", "NonClickbait_Auto"]
categories = ["EC", "ET", "GB", "IS", "LC", "PO", "SO"]
categories = ["EC"]

atricles_per_category = 100

def extract_part1(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        news_title = data['labeledDataInfo']['newTitle']
        news_content = data['sourceDataInfo']['newsContent']
        return news_title, news_content
    
def extract_part2(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        news_title = data['sourceDataInfo']['newsTitle']
        news_content = data['labeledDataInfo']['processSentenceInfo']
        content = ""
        for sentence in news_content:
            content += sentence["sentenceContent"] + " "
        return news_title, content


news_titles = []
news_contents = []
labels = []

folder_path = './data/Part1'

for folder in folders:
    for category in categories:
        path = folder_path+"/"+folder+"/"+category
        count = 0
        for filename in os.listdir(path):
            count+=1
            if count >= (atricles_per_category *2 if folder == "NonClickbait_Auto" else atricles_per_category):
                break
            file_path = os.path.join(path, filename)
            if os.path.isfile(file_path):
                try:
                    title, content = extract_part1(file_path)
                    news_titles.append(title)
                    news_contents.append(content)
                    labels.append([0,1] if folder == "NonClickbait_Auto" else [1,0])
                except (json.JSONDecodeError, KeyError) as e:
                    print(f"Error processing {filename}: {e}")
                    continue

# folder_path = './data/Part2'

# for folder in folders:
#     for category in categories:
#         path = folder_path+"/"+folder+"/"+category
#         count = 0
#         for filename in os.listdir(path):
#             count+=1
#             if count >= atricles_per_category:
#                 break
#             file_path = os.path.join(path, filename)
#             if os.path.isfile(file_path):
#                 try:
#                     title, content = extract_part2(file_path)
#                     news_titles.append(title)
#                     news_contents.append(content)
#                     labels.append([0,1]if folder == "NonClickbait_Auto" else [1,0])
#                 except (json.JSONDecodeError, KeyError) as e:
#                     print(f"Error processing {filename}: {e}")
#                     continue

print("title:", news_titles[:5])
print(len(news_titles))

title: ['의자 전문 브랜드 시디즈, 세계 1위 기저귀 브랜드 팸퍼스와 ‘이것’ 실시… “시디즈의 최초 콜라보”', "인류 최초의 커피 '예멘 모카 마타리'는 언제부터 경작했나?", "'자강두천' 홈플러스-롯데마트, 콩나물 두고 기싸움", 'JP모건체이스, 쫌쫌따리 실적에 주가 부진', "산림경영 적극 참여를 위한 컨설팅 제공, '꿀이네~'"]
298


In [2]:
data = []

for i in range(len(labels)):
    data.append([news_titles[i], news_contents[i], labels[i]])

del news_titles
del news_contents
del labels

random.shuffle(data)

train_titles = []
train_contents = []
labels = []

for d in data:
    train_titles.append(d[0])
    train_contents.append(d[1])
    labels.append(d[2])

In [3]:
print(train_titles[16])
print(train_contents[16])
print(labels[16])

당근마켓, 앱스토어·구글플레이 소셜 부문 1위
당근마켓(공동대표 김용현·김재현)은 앱스토어, 구글플레이 양대 앱 마켓에서 소셜 카테고리 부문 1위에 올랐다고 11일 밝혔다.
당근마켓은 애플 앱스토어와 구글플레이 소셜 카테고리 부문에서 나란히 1위를 기록했다.
이날 앱스토어 '소셜 네트워킹' 무료 앱 카테고리에는 1위 당근마켓, 2위 에브리타임, 3위 카카오톡, 4위 페이스북, 5위 네이버 밴드가 자리하고 있다.
구글플레이 '소셜' 인기차트는 1위 당근마켓, 2위 이프랜드, 3위 인스타그램, 4위 네이버밴드, 5위 트위터 순이다.
당근마켓은 2020년 11월에 와이즈앱 발표 기준 한국인이 가장 많이 사용하는 앱 순위에서 페이스북을 뛰어넘는 기록을 세웠다.
5개월 후인 이듬해 4월에는 인스타그램, 네이버밴드까지 제치고 자주 사용하는 앱 5위권에 오른 바 있다.
당근마켓 관계자는 \"모바일 앱 양대 마켓에서 소셜 부문 1위를 나란히 차지한 것은, 당근마켓이 동네 생활 커뮤니티로 성공적으로 안착한 것으로 볼 수 있다\"며 \"앞으로도 당근마켓은 한계를 짓지 않는 소통의 확장성을 기반으로 지역생활 커뮤니티와 비즈니스의 무한 가능성을 열어가며 하이퍼로컬 혁신을 가속화해 나갈 것\"이라고 말했다.
[0, 1]


In [4]:
cnt=0 # # of clickbait
for label in labels:
    if label == [1,0]:
        cnt+=1
print(cnt)

99


### Preprocessing

In [5]:
okt = Okt()

stop_words_file = open("./stopword.txt").read()
stop_words = stop_words_file.split("\n")

headline_morphed = [okt.morphs(text) for text in train_titles]
body_morphed = [okt.morphs(text) for text in train_contents]

headline_morphed = [[word for word in text if word not in stop_words] for text in headline_morphed]
body_morphed = [[word for word in text if word not in stop_words] for text in body_morphed]

headline_tokenized = [' '.join(text) for text in headline_morphed]
body_tokenized = [' '.join(text) for text in body_morphed]

tokenizer = Tokenizer(num_words=500000)
tokenizer.fit_on_texts(headline_tokenized + body_tokenized)

headline_sequences = tokenizer.texts_to_sequences(headline_tokenized)
body_sequences = tokenizer.texts_to_sequences(body_tokenized)

headline_maxlen = 100
body_maxlen = 1000
headline_padded = pad_sequences(headline_sequences, maxlen=headline_maxlen)
body_padded = pad_sequences(body_sequences, maxlen=body_maxlen)

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

12421


In [ ]:
m = 0
for body in body_sequences:
    m = max(len(body), m)

# for body in body_sequences:
#     if len(body) == m:
#         print(body)
#         print(body_tokenized[body_sequences.index(body)])

print(m)

m = 0
for headline in headline_sequences:
    m = max(len(headline), m)

print(m)

# print(headline_morphed[5])
# print(headline_tokenized[5])
# print(headline_sequences[5])
print(torch.tensor(headline_padded).shape)
print()

1106
25
torch.Size([298, 100])


### Model

In [7]:
class TextBiLSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, bi_lstm_units, max_pool_kernel_size,
                 max_pool_stride, fully_connected_units, dropout_rate, headline_maxlen, body_maxlen):
        super(TextBiLSTMModel, self).__init__()

        # Embedding layers
        self.headline_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.body_embedding = nn.Embedding(vocab_size, embedding_dim)

        # BiLSTM layers
        self.bi_lstm1_headline = nn.LSTM(embedding_dim, bi_lstm_units, bidirectional=True, batch_first=True)
        self.bi_lstm2_headline = nn.LSTM(bi_lstm_units * 2, bi_lstm_units, bidirectional=True, batch_first=True)
        
        self.bi_lstm1_body = nn.LSTM(embedding_dim, bi_lstm_units, bidirectional=True, batch_first=True)
        self.bi_lstm2_body = nn.LSTM(bi_lstm_units * 2, bi_lstm_units, bidirectional=True, batch_first=True)

        # Max Pooling
        self.max_pool = nn.MaxPool1d(kernel_size=max_pool_kernel_size, stride=max_pool_stride, padding=max_pool_kernel_size // 2)

        # Dropout
        self.dropout = nn.Dropout(dropout_rate)

        # Fully connected layers
        headline_output_size = bi_lstm_units * 2 * ((headline_maxlen // max_pool_stride // max_pool_stride) + 1)  # Adjusted for max pooling
        body_output_size = bi_lstm_units * 2 * ((body_maxlen // max_pool_stride // max_pool_stride) + 1)  # Adjusted for max pooling

        self.fc1 = nn.Linear(headline_output_size + body_output_size, fully_connected_units)
        self.fc2 = nn.Linear(fully_connected_units, fully_connected_units)
        self.output = nn.Linear(fully_connected_units, 2)  # 최종 출력은 클래스 수에 맞게 2

    def forward(self, headline_input, body_input):
        # Headline processing
        x = self.headline_embedding(headline_input)
        x, _ = self.bi_lstm1_headline(x)
        x = self.max_pool(x.permute(0, 2, 1)).permute(0, 2, 1)
        x = self.dropout(x)
        x, _ = self.bi_lstm2_headline(x)
        x = self.max_pool(x.permute(0, 2, 1)).permute(0, 2, 1)
        x = self.dropout(x)

        # Body processing
        y = self.body_embedding(body_input)
        y, _ = self.bi_lstm1_body(y)
        y = self.max_pool(y.permute(0, 2, 1)).permute(0, 2, 1)
        y = self.dropout(y)
        y, _ = self.bi_lstm2_body(y)
        y = self.max_pool(y.permute(0, 2, 1)).permute(0, 2, 1)
        y = self.dropout(y)

        # Flatten and concatenate
        x = x.contiguous().view(x.size(0), -1)
        y = y.contiguous().view(y.size(0), -1)
        concatenated = torch.cat((x, y), dim=1)

        # Fully connected layers
        z = F.relu(self.fc1(concatenated))
        z = self.dropout(z)
        z = F.relu(self.fc2(z))
        z = self.dropout(z)

        # Output layer
        output = F.softmax(self.output(z), dim=1)

        return output
    
# 모델 하이퍼파라미터
embedding_dim = 512
bi_lstm_units = 256
max_pool_kernel_size = 10
max_pool_stride = 5
fully_connected_units = 1024
dropout_rate = 0.1
learning_rate = 0.00001

# 모델 생성
model = TextBiLSTMModel(vocab_size, embedding_dim, bi_lstm_units,
                        max_pool_kernel_size, max_pool_stride,
                        fully_connected_units, dropout_rate,
                        headline_maxlen, body_maxlen)

# 옵티마이저와 손실 함수 설정
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# 모델 요약
print(model)

TextBiLSTMModel(
  (headline_embedding): Embedding(12421, 512)
  (body_embedding): Embedding(12421, 512)
  (bi_lstm1_headline): LSTM(512, 256, batch_first=True, bidirectional=True)
  (bi_lstm2_headline): LSTM(512, 256, batch_first=True, bidirectional=True)
  (bi_lstm1_body): LSTM(512, 256, batch_first=True, bidirectional=True)
  (bi_lstm2_body): LSTM(512, 256, batch_first=True, bidirectional=True)
  (max_pool): MaxPool1d(kernel_size=10, stride=5, padding=5, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc1): Linear(in_features=23552, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (output): Linear(in_features=1024, out_features=2, bias=True)
)


In [ ]:
batch_size = 35

headline_input_data = torch.tensor(headline_padded)
body_input_data = torch.tensor(body_padded)

dataset = TensorDataset(headline_input_data, body_input_data, labels)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for headline_input, body_input, true_labels in data_loader:
        # 데이터를 GPU로 이동
        headline_input = headline_input.to(device)
        body_input = body_input.to(device)
        labels = labels.to(device)

        # 옵티마이저 초기화
        optimizer.zero_grad()

        # 모델 예측
        outputs = model(headline_input, body_input).float()
        # print(true_labels[:5])

        # 손실 계산
        loss = criterion(outputs, true_labels)

        # 역전파 및 가중치 업데이트
        loss.backward()
        optimizer.step()

        # 손실 값 저장
        total_loss += loss.item()

    # 에포크별 손실 출력
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(data_loader):.4f}")

print("Training completed!")

tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.]])
tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.]])


KeyboardInterrupt: 

In [ ]:
test_headline_texts = ["바이든 “트럼프 과녁” 해명에 트럼프 지지층 또 발끈", "제2의 크림빵 사건 없게…'한국형 위드마크 공식' 만든다","레노버, 인텔 LTE-A 모뎀 첫 탑재 스마트폰 'P90' 공개","70만원 지원받은 구형 스마트폰 계약해지하면 다 뱉어내?"]
test_body_texts = ["""조 바이든 미국 대통령이 도널드 트럼프 전 대통령에 대한 ‘과녁 정중앙(bullseye)’ 발언을 실수라고 인정했다. 하지만 해명 과정에서 총기의 조준점인 ‘십자선(crosshairs)’을 추가로 언급해 트럼프 열성 지지층의 거센 반발만 불러왔다. 트럼프 피격 사건의 충격 속에 잠잠했던 민주당 내 ‘바이든 용퇴론’이 물밑에서 지속되고 있다는 보도도 나왔다. 바이든 대통령은 15일(현지시간) NBC방송과의 인터뷰에서 ‘과녁 정중앙’ 발언과 관련해 “나는 십자선이라고 말하지 않았다. 과녁 정중앙이라고 했다. 그 단어를 사용한 것은 실수였다”며 “트럼프에게 집중하고, 그의 행동에 집중하라는 의미였다”고 밝혔다. 앞서 바이든 대통령은 지난 8일 후원자들과의 전화 통화에서 “트럼프를 과녁 정중앙에 놓아야 할 때”라고 말했다. 이 발언은 지난달 27일 대선 TV토론 참패로 자신에게 쏟아지던 민주당 내 비판 여론을 트럼프 쪽으로 돌려놓으려는 시도로 해석됐다. 하지만 트럼프가 지난 13일 펜실베이니아주 버틀러 유세에서 총격을 당한 뒤 ‘과녁 정중앙’ 발언은 바이든 대통령에게 역풍으로 돌아갔다. 트럼프 열성 지지층은 소셜미디어에서 “바이든이 암살 표적을 지목한 것”이라는 식의 주장을 퍼뜨렸고, 이 주장에 동조하지 않는 공화당 인사들도 “바이든의 이런 수사(修辭)가 결국 암살 시도를 유발한 것”이라고 비판했다. 이날 ‘십자선’을 언급한 바이든 대통령의 해명도 지적을 받았다. 과녁 정중앙과 십자선이 목표물과 조준점의 차이일 뿐 모두 총기를 연상시킨다는 이유에서다. 2016년과 2020년 트럼프 대선 캠프 고문을 지낸 스티브 코르테스는 이날 엑스(옛 트위터)에서 “바이든의 발언은 결국 트럼프를 겨냥한 사실을 후회하지 않는다는 의미”라고 지적했다. 바이든 대통령은 트럼프 피격 이후 당과 대선 캠프에 공격적인 언행을 자제하라고 당부했지만, 자신은 이날 인터뷰에서 공세를 이어갔다. 그는 “트럼프가 TV토론에서 했던 거짓말에 집중해 보라. 나는 취임 첫날 독재자가 되겠다고 말하지도, 선거 결과를 거부하지도 않았다”고 강조했다. 트럼프 피격 사건이 대선에 미칠 영향에 대해서는 “나도 모르고 당신(진행자)도 모른다”고 말했다. 바이든 대통령은 공화당 전당대회에서 부통령 후보로 선출된 J.D 밴스 상원의원에 대해선 “트럼프의 복제인간”이라고 비판했다. CNN은 바이든 대통령의 이날 발언들에 대해 “물러날 생각이 없다고 분명하게 보여준 것”이라며 “최근 수일간 잠잠했던 후보 사퇴론이 당내에서 지속되고 있다”고 보도했다. 최근 민주당 내에선 여론조사 전문가 스탠리 그린버그의 대선 패배 전망 보고서가 떠돌고 있으며, 이를 본 의원들은 “충격적”이라거나 “모두 잃을 것”이라고 반응했다고 CNN은 전했다. 그린버그는 11월 대선에서 바이든 대통령이 질 것이며 이는 상하원 선거에 출마한 민주당 후보들에게 큰 피해를 줄 것이라는 견해를 보고서에 담았다.""",
                   """ "레노버가 세계 최대 가전전시회 'CES 2015'에서 인텔 LTE-A(롱텀에볼루션 어드밴스드) 모뎀을 탑재한 첫 스마트폰을 공개했다고 6일 밝혔다.\n레노버 P90은 최신 64비트 인텔 아톰 Z3560 프로세서와 인텔의 XMM 7262 모뎀을 탑재했다.\nXMM7262는 최대 다운로드 속도 300Mbps의 LTE-A가 가능하다.\nP90은 4000mAh(밀리암페어아워) 배터리를 탑재했으며 5.5형 풀HD 디스플레이를 채용했다.\n또 어두운 곳에서도 훌륭한 사진을 촬영할 수 있도록 최첨단 적층식 센서(stacking sensor)와 광학식 손떨림 보정 기술을 적용한 1300만화소 카메라를 탑재했다.\n전면 카메라는 500만화소로 제스처 컨트롤 기능을 갖추고 있다.\nP90은 펄 화이트(Pearl White), 오닉스 블랙(Onyx Black), 라바 레드(Lava Red) 등 3가지 색상으로 출시될 예정이다.\n레노버 바이브 X2 프로 리미티드 에디션은 가볍고 세련된 디자인의 풀 메탈 바디를 지녔다.\n64비트 퀄컴 스냅드래곤 옥타코어 프로세서와 5.3형 풀HD 디스플레이를 탑재했다.\n레노버 X2 프로는 듀얼 1300만 화소 오토 포커스 카메라를 탑재했다.\n특히 바이브 엑스텐션 셀피 플래시를 이용해 완벽한 셀카를 찍을 수 있다.\n레노버 바이브 익스텐션 셀피 플래시는 조명과 상관 없이 모든 순간을 포착할 수 있는 액세서리이다.\n포켓 사이즈의 플래시를 오디오 잭에 꽂으면 조명과 셔터 싱크를 손쉽게 조절할 수 있어서 언제 어디에서나 밝고 자연스러운 모습을 사진으로 남길 수 있다.\n셀피 플래시는 8개 산란형 LED를 이용하며 어두울 때에도 자연스러운 색조를 위해 필요한 광원을 보충한다.\n레노버 바이브 밴드 VB10은 손쉽게 인터넷에 접속할 수 있도록 설계됐다.\nE 잉크 디스플레이, 화면과 블루투스를 켜두고도 최장 7일간 작동하는 배터리, 피트니스 기능, 전화 알림, 방수 기능 등을 갖췄다.\n바이브 밴드의 E 잉크 디스플레이를 적용해 태양광에서도 화면을 눈부심 없이 깨끗하게 볼 수 있다.\n전화, SMS(문자), 페이스북, 트위터 등 알림이 오면 최대 150자까지 내용을 즉시 확인할 수 있다.\n레노버 바이브 밴드 VB10은 모든 안드로이드 또는 iOS 모바일 디바이스와 연동된다.\n다만 이날 공개한 레노버 스마트폰과 모바일 액세서리의 국내 출시 계획은 미정이다.\n샤오 타오 레노버 모바일 비즈니스 부사장은 \\\"점차 모바일 라이프스타일이 일상화되면서 소비자들은 얇고 스타일리시한 디자인으로 성능과 오랜 시간 동안 지속되는 배터리 수명을 모두 제공하는 디바이스를 원하고 있다\\\"ㅁ \\\"초고속 처리 속도, 혁신적인 새로운 기능, 더 긴 배터리 수명을 제공한다\\\"고 말했다.""",
                   """ "레노버가 세계 최대 가전전시회 'CES 2015'에서 인텔 LTE-A(롱텀에볼루션 어드밴스드) 모뎀을 탑재한 첫 스마트폰을 공개했다고 6일 밝혔다.\n레노버 P90은 최신 64비트 인텔 아톰 Z3560 프로세서와 인텔의 XMM 7262 모뎀을 탑재했다.\nXMM7262는 최대 다운로드 속도 300Mbps의 LTE-A가 가능하다.\nP90은 4000mAh(밀리암페어아워) 배터리를 탑재했으며 5.5형 풀HD 디스플레이를 채용했다.\n또 어두운 곳에서도 훌륭한 사진을 촬영할 수 있도록 최첨단 적층식 센서(stacking sensor)와 광학식 손떨림 보정 기술을 적용한 1300만화소 카메라를 탑재했다.\n전면 카메라는 500만화소로 제스처 컨트롤 기능을 갖추고 있다.\nP90은 펄 화이트(Pearl White), 오닉스 블랙(Onyx Black), 라바 레드(Lava Red) 등 3가지 색상으로 출시될 예정이다.\n레노버 바이브 X2 프로 리미티드 에디션은 가볍고 세련된 디자인의 풀 메탈 바디를 지녔다.\n64비트 퀄컴 스냅드래곤 옥타코어 프로세서와 5.3형 풀HD 디스플레이를 탑재했다.\n레노버 X2 프로는 듀얼 1300만 화소 오토 포커스 카메라를 탑재했다.\n특히 바이브 엑스텐션 셀피 플래시를 이용해 완벽한 셀카를 찍을 수 있다.\n레노버 바이브 익스텐션 셀피 플래시는 조명과 상관 없이 모든 순간을 포착할 수 있는 액세서리이다.\n포켓 사이즈의 플래시를 오디오 잭에 꽂으면 조명과 셔터 싱크를 손쉽게 조절할 수 있어서 언제 어디에서나 밝고 자연스러운 모습을 사진으로 남길 수 있다.\n셀피 플래시는 8개 산란형 LED를 이용하며 어두울 때에도 자연스러운 색조를 위해 필요한 광원을 보충한다.\n레노버 바이브 밴드 VB10은 손쉽게 인터넷에 접속할 수 있도록 설계됐다.\nE 잉크 디스플레이, 화면과 블루투스를 켜두고도 최장 7일간 작동하는 배터리, 피트니스 기능, 전화 알림, 방수 기능 등을 갖췄다.\n바이브 밴드의 E 잉크 디스플레이를 적용해 태양광에서도 화면을 눈부심 없이 깨끗하게 볼 수 있다.\n전화, SMS(문자), 페이스북, 트위터 등 알림이 오면 최대 150자까지 내용을 즉시 확인할 수 있다.\n레노버 바이브 밴드 VB10은 모든 안드로이드 또는 iOS 모바일 디바이스와 연동된다.\n다만 이날 공개한 레노버 스마트폰과 모바일 액세서리의 국내 출시 계획은 미정이다.\n샤오 타오 레노버 모바일 비즈니스 부사장은 \\\"점차 모바일 라이프스타일이 일상화되면서 소비자들은 얇고 스타일리시한 디자인으로 성능과 오랜 시간 동안 지속되는 배터리 수명을 모두 제공하는 디바이스를 원하고 있다\\\"ㅁ \\\"초고속 처리 속도, 혁신적인 새로운 기능, 더 긴 배터리 수명을 제공한다\\\"고 말했다.""",
                    "정부가 출시 후 15개월 지난 구형 휴대폰에 한해 위약금 총액을 일정액 이상 넘을 수 없도록 하는 '위약금 상한제' 도입을 검토 중이다.\n출시 후 15개월이 지난 단말기의 경우, 정부의 지원금 상한액(최고 30만원) 규제 대상에서 제외되면서 소비자 지원금이 대폭 상향 조정되는데, 계약 해지시 소비자들의 위약금 부담도 그만큼 커질 수 있다는 지적 때문이다.\n6일 미래부 관계자는 \\\"15개월 경과된 단말기 지원금이 최고 70만~80만원 크게 오르면서 이에 따른 위약금 부담이 클 수밖에 없다\\\"며 \\\"소비자들의 위약금 부담 경감 차원에서 위약금 상한액을 두는 방안에 대해 이통사들과 협의 중\\\"이라고 밝혔다.\n위약금 상한제란 가입자가 약정기간 중 계약을 해지할 경우 이통사에 내야 하는 위약금(반환액) 총액에 상한을 두겠다는 것. 현재 이동통신 3사의 위약금 제도는 계약 체결 후 1개월~6개월 지난 뒤 매달 위약금을 서서히 경감해주는 구조다.\n하지만 출시 후 15개월이 경과된 구형 단말기의 경우 지원금이 수직 상승하는 탓에 계약 해지 시 일시 지불해야 될 비용부담이 그만큼 클 수 밖에 없다.\n가령, 이달부터 지원금 상한제 대상 품목에서 제외된 '삼성 갤럭시노트3'의 경우, 최대 보조금 평균(최고 요금제 기준)이 전달 28만4000원에서 75만2000원으로 급상승했다.\nLG G2 역시 최대 보조금 평균이 40만9000원에서 57만7000원으로 껑충 뛰었다.\n이 상황에서 구형 단말기임에도 계약 초기에 단말기를 분실했거나 피치 못할 사정으로 계약을 해지할 경우 출고가에 버금가는 위약금 폭탄을 안을 수 있다는 얘기다.\n그러나 이동통신사들은 제도 도입에 난색을 보이고 있어 이 제도가 시행될 지 여부는 불투명하다.\n위약금 상한액을 둘 경우 이 제도를 악용해 중고폰으로 되팔아 이익을 남기는 '폰테크'가 성행하는 등 부작용도 만만치 않을 것이라는 우려 때문이다.\n미래부 관계자는 \\\"15개월이 지난 단말기의 경우, 출고가가 떨어 져야 하는 게 바람직하지만 사업자들간의 이해관계가 얽히면서 지원금 상승으로 이어지고 있다는 게 문제\\\"라며 \\\"합리적인 수준에서 위약금 제도가 보완될 수 있도록 이통사들과 협의해 나갈 계획\\\"이라고 밝혔다."]

# Tokenize texts
test_headline_tokenized = [' '.join(okt.morphs(text)) for text in test_headline_texts]
test_body_tokenized = [' '.join(okt.morphs(text)) for text in test_body_texts]

# Convert texts to sequences
test_headline_sequences = tokenizer.texts_to_sequences(test_headline_tokenized)
test_body_sequences = tokenizer.texts_to_sequences(test_body_tokenized)

# Pad sequences
test_headline_padded = pad_sequences(test_headline_sequences, maxlen=headline_maxlen)
test_body_padded = pad_sequences(test_body_sequences, maxlen=body_maxlen)

# print(len(test_body_sequences[0]))

predictions = model.predict([test_headline_padded, test_body_padded])

for predict in predictions:
    print("possibility of clickbait: "+str(round(float(predict[0])*100,2))+"%")